----------------------------------------------------------------------------
Copyright (c) 2015--, platypus development team.

Distributed under the terms of the BSD License.

The full license is in the file [COPYING.txt](https://raw.githubusercontent.com/biocore/Platypus-Conquistador/master/COPYING.txt), distributed with this software.

----------------------------------------------------------------------------

## Chasing the Platypus via MG-RAST

This tutorial covers downloading all public MG-RAST datasets; parsing those files to search for a specific taxa of interest, in our case platypus (Ornithorhynchus), Salmonella, dodo bird (Raphus) and Tasmanian tiger( Thylacinus); and finding the geographycal location of the samples that match those samples, via Google. Note that you need [BeautifulSoup](http://www.crummy.com/software/BeautifulSoup/#Download/) installed.

## Downloading all public MG-RAST datasets

**Before you continue:** This step is extremelly slow (~24 hrs.) and network intensive. Thus to facilitate this step you could download the files from April 22, 2015 [ftp://ftp.microbio.me/pub/platypus/mgrast_reports_042215.tgz](ftp://ftp.microbio.me/pub/platypus/mgrast_reports_042215.tgz). If you place them in the same folder that you have this IPython Notebook the code will simply download the latests samples.

The first step is to crawl MG-RAST's website to search for all public datasets, downloading both the data (in HTML format) and the metadata (tab separated) for each study and store them locally. To avoid having hundreds of files in a single directory the script will group them in folders by the first 3 characters of its name. Note that MG-RAST calls each sample a project, here we will call them samples.

In [ ]:
# Downloading previous version
!curl -O ftp://ftp.microbio.me/pub/platypus/mgrast_reports_042215.tgz
!tar zxvf mgrast_reports_042215.tgz

In [ ]:
from BeautifulSoup import BeautifulSoup
from urllib2 import urlopen, URLError
from os.path import isfile, exists, join
from os import makedirs
from socket import timeout as TOError

mgrast_analysis = 'http://metagenomics.anl.gov/metagenomics.cgi?page=Analysis'
mgrast_metagenome = ('http://metagenomics.anl.gov/metagenomics.cgi?page='
                     'MetagenomeOverview&metagenome=')
mgrast_metadata = ('http://metagenomics.anl.gov/metagenomics.cgi?page='
                   'MetagenomeProject&action=download_md&filetype=text&'
                   'filename=mgm%s.metadata.txt')

page = urlopen(mgrast_analysis)
soup = BeautifulSoup(page.read())
count = 0
all_inputs = soup.findAll('input', {'id': 'list_select_data_0'})[0]
print '====> Got Analysis page results'
for maingroups in all_inputs['value'].encode('utf-8').split('||'):
    for mg in maingroups.split('@'):
        sample = mg.split('##')[0].split('~#')[0]
        # will group by the first 3 digits of the project name
        dirname = sample[:3]

        sample_fp = join(dirname, 'sample_' + sample + '.txt')
        map_fp = join(dirname, 'map_' + sample + '.txt')
        
        count = count + 1
        if count % 50 == 0:
            print "====> Processed %d samples" % count
        # continue if sample is blank or file already exists
        if sample == '' or (isfile(sample_fp) and isfile(map_fp)):
            continue

        print 'Processing (%d): %s %s' % (count, sample, mg)

        if not exists(dirname):
            makedirs(dirname)

        # visiting the Overview page to download counts
        try:
            page_sample = urlopen(mgrast_metagenome + sample, timeout=40)
            page_mapping = urlopen(mgrast_metadata % sample, timeout=40)
        except (TOError, URLError), e:
            # Timeout
            print e
            count = count - 1
            continue
        
        open(sample_fp, 'w').write(page_sample.read())
        open(map_fp, 'w').write(page_mapping.read())

## Finding taxa

**Before you continue:** This step is not as slow as the previous one but still a bit slow (~1 hr). If you only want to reproduce the results from the paper you can download this file: [ftp://ftp.microbio.me/pub/platypus/mgrast_search_results.tgz](ftp://ftp.microbio.me/pub/platypus/mgrast_search_results.tgz)

Now that we have all MG-RAST files in our local folder we can search for any taxa of interest. Note that the HMTL filess that we downloaded in the previous step have all data information that MG-RAST provides from a sample, for example: multilevel taxonomic and functional assignments. Thus, in the next step you can search by any "string" that fits in those categories. The output of this section is tab separated text file named as the query (lower case) with three columns: the name found, the sample name and counts of the successful search on that sample.

In [ ]:
# Downloading previous version
!curl -O ftp://ftp.microbio.me/pub/platypus/mgrast_search_results.tgz
!tar zxvf mgrast_search_results.tgz

In [ ]:
from BeautifulSoup import BeautifulSoup
from os import listdir, makedirs
from os.path import isdir, isfile, join, basename, exists

queries = ['Ornithorhynchus', 
           'Salmonella', 
           'Raphus', 
           'Thylacinus']

queries = [q.lower() for q in queries]

queries_fp = [open(q + '.txt', 'w') for q in queries]

# adding a " at the beginning of the queries so we do exact matching
queries = ['"%s' % q for q in queries]

# looping in local dir to find downloaded folders
# all mgrast created folders are 3 chars long and start with a 4
dirs = [d for d in listdir('./')
        if len(d) == 3 and d.startswith('4') and isdir(d)]

count_files = 0
for d in dirs:
    # getting files in current directory
    files = [join(d, f) for f in listdir(d) if f.startswith('sample_')]
    for sample in files:
        count_files = count_files + 1
        if count_files % 250 == 0:
            print 'Processed %d files' % count_files
        sample_id = basename(f)[len('sample_'):-len('.txt')]
        mapping = join(d, 'map_%s.txt' % sample_id)

        # validating that we have both the sample and the metadata file for
        # the given project
        if not isfile(sample) or not isfile(mapping):
            raise ValueError('Missing files %s, %s' % (sample, mapping))

        soup = BeautifulSoup(open(sample, 'U').read())
        for s in soup.findAll('script'):
            s = str(s).lower()
            
            for q, qfp in zip(queries, queries_fp):
                if q in s:
                    target = [t for t in s.split('\n') if q in t][0]
                    name, counts = target.split(',')
                    name = name.split('data.addrow([')[1].strip('"')
                    counts = counts[:counts.find(']')]
                    qfp.write('%s\t%s\t%s\n' % (name, sample_id, counts))
                
_ = [q.close() for q in queries_fp]

**Note:** We suggest reviewing the resulting files, in specific check that the first column actually matches your query. Depending on the how broad the term is, it could have hits to other taxa.

## Finding geographical location of samples and summarizing results

The first step is to generate a dictionary of dictonaries containing the counts per country of all the files generated in the previous step. Note that you need to feed which files you want to parse. To find the country of origin of the samples we will use the latitute and longitude values from the metadata. The speed on this step depends a lot on the number of hits and the diversity of countries in the results. For the example it took a little over 1 hr to run. Finally, remember that we are relying on an online (Google) service so the country summaries are based on its availability and successful reply.

In [ ]:
from urllib2 import urlopen
import json

from os import listdir
from os.path import isdir, isfile, join
from time import sleep

           
in_fps = ['ornithorhynchus.txt', 
          'raphus.txt', 
          'salmonella.txt', 
          'thylacinus.txt']

# from http://stackoverflow.com/a/20169528/4228285
def getplace(lat, lon):
    url = ("http://maps.googleapis.com/maps/api/geocode/json?"
           "latlng=%s,%s&sensor=false" % (lat, lon))
    # googleapis blocks requests that happen too fast so
    # making sure to wait for 5 seconds before quering the
    # api
    sleep(5)
    v = urlopen(url).read()
    j = json.loads(v)

    if len(j['results'])==0:
        return 'Undefined'

    components = j['results'][0]['address_components']
    country = town = None
    for c in components:
        if "country" in c['types']:
            country = c['long_name']

    return str(country)

print 'starting processing'
sample_country = {}
sample_country_counts = {fps:{} for fps in in_fps}
for fps in in_fps:
    print "Processing file %s" % fps
    for line in open(fps, 'r'):
        _, sample, counts = line.strip().split('\t')
        counts = int(counts)
        
        if sample not in sample_country:
            mapping = join(sample[:3], 'map_%s.txt' % sample)

            if not isfile(mapping):
                raise ValueError("%s doesn't exist!!" % (mapping))

            # getting lat/long and quering Google for country
            lat = ''
            lon = ''
            for line in open(mapping, 'r'):
                line = line.split('\t')

                # sometimes the lat/lon have different comma separated values
                # will only use the first one
                if line[0] == 'Sample' and line[1] == 'longitude':
                    lon = float(line[2].strip().split(',')[0].strip())
                elif line[0] == 'Sample' and line[1] == 'latitude':
                    lat = float(line[2].strip().split(',')[0].strip())

                if not lat or not lon:
                    country = 'Undefined'
                else:
                    country = getplace(lat, lon)
            
            sample_country[sample] = country
        else:
            country = sample_country[sample]
        
        if country not in sample_country_counts[fps]:
            sample_country_counts[fps][country] = 0
            
        sample_country_counts[fps][country] += counts

Now, let's save the results in a single file. The example files can be downloaded from [ftp://ftp.microbio.me/pub/platypus/mgrast_countries_results.tgz](ftp://ftp.microbio.me/pub/platypus/mgrast_countries_results.tgz).

In [ ]:
# Downloading previous version
!curl -O ftp://ftp.microbio.me/pub/platypus/mgrast_countries_results.tgz
!tar zxvf mgrast_countries_results.tgz

In [ ]:
from operator import itemgetter


for fps, results in sample_country_counts.items():
    sorted_results = sorted(results.items(),
                            key=itemgetter(1)) 
    fps_out = open(fps[:-len('.txt')] + '_countries.txt', 'w')
    for country, count in sorted_results:
        fps_out.write("%s\t%d\n" % (country, count))
    fps_out.close()

Finally, we can use those results to generate some visualizations.

In [1]:
from IPython.display import Javascript, display
from jinja2 import Template
from pycountry import countries

def make_map(data_df, filename='Mislabelings'):
    """Create an interactive map with DataMaps
    
    Parameters
    ----------
    data_df : pd.DataFrame
        DataFrame with Country, Counts and Color
    name : str
        Name of the mislabeled counts
        
    Returns
    -------
    IPython display object
        HTML display object.
    """
    name = filename.split('_')[0].title()

    # format a JavaScript dictionary
    country_data = []
    for _, a in data_df.iterrows():
        try:
            country = countries.get(name=a.Country).alpha3
        except KeyError:
            continue
        country_data.append("'%s': {count: %s, fillKey: '%s'}," % (country, a.Counts, a.Color))
    country_data = '\n'.join(country_data)
    
    # this is a JavaScript dictionary of colors ... datamaps has an odd API
    color_data = '\n'.join(["'%s': '%s'," % (a, a) for a in df.Color.unique()])

    js_template = """
        $(element).append('<h1>Thylacinus</h1');
        $(element).append('<div id="{{ name }}" style="position: relative; width:500px; height:300px;"></div>');

        require.config({
                        paths: {d3: "https://cdn.rawgit.com/ElDeveloper/a6fc14f8993a1b8ad843/raw/ca8d8c12125cd93ccf6902c4dacce473e4cf2c3b/d3.v3.min",
                        topojson: "https://cdn.rawgit.com/ElDeveloper/7d4f4a12683b03008435/raw/7ab04a76821117541a86764036e41edb00fae4f3/topojson.v1.min",
                        datamaps: "https://cdn.rawgit.com/ElDeveloper/61b29679961d29e94db1/raw/d05e3e44485a414b4f32b16457c5bcef6de1a6a7/datamaps.all"}
        });

        require(["d3", "topojson", "datamaps"], function(d3, topojson, datamaps) {
            var rawData = { {{ country_data }} };
            var map = new  $({{ name }}).datamaps({
                    fills: {
                        defaultFill: 'gray',
                        red: 'red',
                        {{ color_data }}
                    },
                    geographyConfig: {
                        // highlightOnHover: false,
                        highlightBorderColor: '#bada55',
                        popupTemplate: function(geography, data) {
                            if (data !== null){
                                return '<div class="hoverinfo">There were '+ data.count + ' {{ name }} found here' + '</div>';
                            }
                        },
                        highlightBorderWidth: 1
                      },
                    data: rawData

                    });
        });
    """
    temp = Template(js_template)
    output = temp.render(country_data=country_data, color_data=color_data,
                         name=name)
    display(Javascript(output))

In [2]:
from matplotlib.colors import rgb2hex
from pycountry import countries

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

fps = ['ornithorhynchus_countries.txt',
       'raphus_countries.txt',
       'salmonella_countries.txt',
       'thylacinus_countries.txt']

for fp in fps:
    df = pd.read_csv(fp, sep='\t', names=['Country', 'Counts'])

    # normalize the range for the colors in a log scale
    log_counts = np.log(df.Counts)
    df['Color'] = map(rgb2hex, plt.cm.YlGn(log_counts/log_counts.max()))
    
    make_map(df, filename=fp)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>